# CS-484: Intro to ML - Assignment 3


### Author: Jorge Gonzalez Lopez
### CWID: A20474413
### Semester: Spring 2021

In [1]:
# IMPORT LIBRARIES

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as stats
import scipy
from itertools import combinations


## Question 2

In [3]:
df = pd.read_csv('claim_history.csv')

In [4]:
columns = df[['CAR_TYPE','OCCUPATION','EDUCATION', 'CAR_USE']]

In [5]:
#print('Total number of samples', len(columns),'\n')
#print('Percentage of Target Values: \n')

probs_targ = columns['CAR_USE'].value_counts(normalize=True)

In [6]:
E_target = np.sum(-probs_targ * np.log2(probs_targ))
print('a) The entropy of the root node is: ', E_target)

a) The entropy of the root node is:  0.9489621493401781


In [7]:
comb_occ = []
for i in range(1,len(columns["OCCUPATION"].unique())):
    comb_occ+=list(combinations(columns["OCCUPATION"].unique(),i))

comb_type = []
for i in range(1,len(columns["CAR_TYPE"].unique())):
    comb_type+=list(combinations(columns["CAR_TYPE"].unique(),i))
    
comb_edu = [("Below High School",),("Below High School","High School",),("Below High School","High School","Bachelors",),("Below High School","High School","Bachelors","Masters",)]


In [8]:
def get_optimal_split(df,column,target,combinations):
    
    best_split = []
    Gain = 0
    split_entropy = 1
    
    #Get entropy of target value of all the samples in the dataset df
    probs_targ = df[target].value_counts(normalize=True)
    E_target = np.sum(-probs_targ * np.log2(probs_targ))    
    
    #For all possible combinations:
    for comb in combinations:
           
        #transform the combination into an array -> left split
        split_1 = []
        for values in comb:
            split_1.append(values)
            
        #Get the array for the values of the column no included in the combination -> right split
        split_2 = [ele for ele in df[column].unique() if ele not in split_1] 
        
        # Subgroup 1: dataset which values of column are contain in split1
        # Get probabilities of being Private / Commercial for subgroup1
        sub1 = df[df[column].isin(split_1)][target].value_counts(normalize=True)
        #Entropy of subgroup1
        E_1 = np.sum(-sub1 * np.log2(sub1))
        
        # Subgroup 2: dataset which values of column are contain in split2
        # Get probabilities of being Private / Commercial for subgroup2
        sub2 = df[df[column].isin(split_2)][target].value_counts(normalize=True)
        #Entropy of subgroup2
        E_2 = np.sum(-sub2 * np.log2(sub2))
        
        # Total number of samples in both subgroups
        T1 = len(df[df[column].isin(split_1)][target])
        T2 = len(df[df[column].isin(split_2)][target])
        
        # Calculate the split entropy (both entropies scale by the probability of ocurrence)
        E_s = E_1 * T1 / (T1+T2) + E_2 * T2 / (T1+T2)
        
        # Check and Store the split that gets the lower split entropy
        if E_s < split_entropy:
            Gain = E_target-E_s
            best_split = [split_1, split_2]
            split_entropy = E_s
            
    return best_split, split_entropy, Gain
    

In [9]:
print('\nb) Left Split, Right Split, Entropy, Gain\n')


b) Left Split, Right Split, Entropy, Gain



In [10]:
print(get_optimal_split(columns,'CAR_TYPE','CAR_USE',comb_type))

([['Minivan', 'SUV', 'Sports Car'], ['Van', 'Panel Truck', 'Pickup']], 0.7684152303050842, 0.1805469190350939)


In [11]:
best_split, ent, gain = get_optimal_split(columns,'OCCUPATION','CAR_USE',comb_occ)
print(best_split, ent, gain)

[['Blue Collar', 'Unknown', 'Student'], ['Professional', 'Manager', 'Clerical', 'Doctor', 'Lawyer', 'Home Maker']] 0.7125832535737261 0.236378895766452


In [12]:
print(get_optimal_split(columns,'EDUCATION','CAR_USE',comb_edu))

([['Below High School'], ['Doctors', 'High School', 'Bachelors', 'Masters']], 0.9356142508258438, 0.013347898514334267)


In [13]:
print('C) OCCUPATION is the feature selected for splitting the first layer')

C) OCCUPATION is the feature selected for splitting the first layer


In [14]:
print('\n Left Branch')

# DATASET LEFT BRANCH: samples that meet the split condition
Left_branch = columns[columns['OCCUPATION'].isin(best_split[0])]


 Left Branch


In [15]:
print(get_optimal_split(Left_branch,'CAR_TYPE','CAR_USE',comb_type))

([['Minivan', 'SUV', 'Sports Car'], ['Van', 'Panel Truck', 'Pickup']], 0.7725782837913744, 0.08232012008258416)


In [16]:
comb_occ2 = []
for i in range(1,len(best_split[0])):
    comb_occ2+=list(combinations(best_split[0],i))

print(get_optimal_split(Left_branch,'OCCUPATION','CAR_USE',comb_occ2))

([['Student'], ['Blue Collar', 'Unknown']], 0.8042192219461466, 0.050679181927811956)


In [17]:
best_split_l, ent, gain = get_optimal_split(Left_branch,'EDUCATION','CAR_USE',comb_edu)

print(best_split_l, ent, gain)

[['Below High School'], ['High School', 'Bachelors', 'Doctors', 'Masters']] 0.6670194998377932 0.18787890403616536


In [18]:
print('\n Right Branch')

# DATASET RIGHT BRANCH: samples that do not meet the split condition
right_branch = columns[columns['OCCUPATION'].isin(best_split[1])]


 Right Branch


In [19]:
best_split_r, ent, gain = get_optimal_split(right_branch,'CAR_TYPE','CAR_USE',comb_type)

print(best_split_r, ent, gain)

[['Minivan', 'SUV', 'Sports Car'], ['Van', 'Pickup', 'Panel Truck']] 0.3274450052616846 0.3001463166908794


In [20]:
comb_occ2 = []
for i in range(1,len(best_split[1])):
    comb_occ2+=list(combinations(best_split[1],i))

print(get_optimal_split(right_branch,'CAR_TYPE','CAR_USE',comb_occ2))

([['Professional'], ['Minivan', 'Van', 'SUV', 'Sports Car', 'Pickup', 'Panel Truck']], 0.627591321952564, 0.0)


In [21]:
print(get_optimal_split(right_branch,'CAR_TYPE','CAR_USE',comb_edu))

([['Below High School'], ['Minivan', 'Van', 'SUV', 'Sports Car', 'Pickup', 'Panel Truck']], 0.627591321952564, 0.0)


In [22]:
print('\nTerminal Nodes')


Terminal Nodes


In [23]:
# LAST DATASETS: 
# samples that meet the split condition
left_left = Left_branch[Left_branch['EDUCATION'].isin(best_split_l[0])]
# samples that do not meet the split condition
left_right = Left_branch[Left_branch['EDUCATION'].isin(best_split_l[1])]

In [24]:
# LAST DATASETS: 
# samples that meet the split condition
right_left = right_branch[right_branch['CAR_TYPE'].isin(best_split_r[0])]
# samples that do not meet the split condition
right_right = right_branch[right_branch['CAR_TYPE'].isin(best_split_r[1])]

In [25]:
def terminal_nodes(df, target):
    #Get the total number of Private / Commercial (target categories) samples in df
    count = df[target].value_counts()
    #Get the probability of Private / Commercial (target categories) in df
    probs_targ = df[target].value_counts(normalize=True)
    # Calculate the entropy
    E = np.sum(-probs_targ * np.log2(probs_targ))
    return E, count, probs_targ

In [26]:
E, count, prob = terminal_nodes(left_left, 'CAR_USE')
print('\n1. Terminal node: Entropy: ', E, '. Private: ' ,count[0], '. Commercial: ' ,count[1],'. Total: ' ,count[0]+count[1], 'Probabilties of Private: ', prob[0],'Probabilties of Commercial: ', prob[1] )
E, count, prob = terminal_nodes(left_right, 'CAR_USE')
print('\n2. Terminal node: Entropy: ', E, '. Private: ' ,count[0], '. Commercial: ' ,count[1],'. Total: ' ,count[0]+count[1], 'Probabilties of Private: ', prob[0],'Probabilties of Commercial: ', prob[1] )
E, count, prob = terminal_nodes(right_left, 'CAR_USE')
print('\n3. Terminal node: Entropy: ', E, '. Private: ' ,count[0], '. Commercial: ' ,count[1],'. Total: ' ,count[0]+count[1], 'Probabilties of Private: ', prob[0],'Probabilties of Commercial: ', prob[1] )
E, count, prob = terminal_nodes(right_right, 'CAR_USE')
print('\n4. Terminal node: Entropy: ', E, '. Private: ' ,count[0], '. Commercial: ' ,count[1],'. Total: ' ,count[0]+count[1], 'Probabilties of Private: ', prob[0],'Probabilties of Commercial: ', prob[1] )



1. Terminal node: Entropy:  0.8304276080710689 . Private:  607 . Commercial:  216 . Total:  823 Probabilties of Private:  0.7375455650060754 Probabilties of Commercial:  0.2624544349939247

2. Terminal node: Entropy:  0.6226204001098349 . Private:  2559 . Commercial:  470 . Total:  3029 Probabilties of Private:  0.8448332783096731 Probabilties of Commercial:  0.15516672169032683

3. Terminal node: Entropy:  0.056791153992247115 . Private:  4564 . Commercial:  30 . Total:  4594 Probabilties of Private:  0.9934697431432303 Probabilties of Commercial:  0.006530256856769699

4. Terminal node: Entropy:  0.9973716177249364 . Private:  984 . Commercial:  872 . Total:  1856 Probabilties of Private:  0.5301724137931034 Probabilties of Commercial:  0.4698275862068966


## Question 3

In [30]:
df = pd.read_csv('sample_v10.csv')
df['y'] = df['y'].astype('category')

In [31]:
print('a) The frequency table of the categorical target field is: ')
print(df['y'].value_counts())


a) The frequency table of the categorical target field is: 
3    4194
2    3532
1    2274
Name: y, dtype: int64


In [32]:
# Backward Selection
# Consider Model 0 is Origin = Intercept + DriveTrain + Weight
y = df['y']
X = df.drop(['y'], axis=1)

columns = X.columns

X = stats.add_constant(X, prepend=True)

DF1 = np.linalg.matrix_rank(X) * (len(y.value_counts()) - 1)

logit = stats.MNLogit(y, X)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK1 = logit.loglike(thisParameter.values)

print(thisFit.summary())
print("Model Log-Likelihood Value =", LLK1)
print("Number of Free Parameters =", DF1)

Optimization terminated successfully.
         Current function value: 0.195606
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9978
Method:                           MLE   Df Model:                           20
Date:                Sun, 07 Mar 2021   Pseudo R-squ.:                  0.8170
Time:                        18:24:15   Log-Likelihood:                -1956.1
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0165      0.087     11.636      0.000       0.845       1.188
x1            -1.1172      0

In [39]:
removed_columns = []
best_model = []
i = 1

while True:
    y = df['y']
    X = df.drop(['y'], axis=1)
    X = X.drop(removed_columns, axis = 1)
    
    columns = X.columns
    
    X = stats.add_constant(X, prepend=True)

    DF1 = np.linalg.matrix_rank(X) * (len(y.value_counts()) - 1)

    logit = stats.MNLogit(y, X)
    thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
    thisParameter = thisFit.params
    LLK1 = logit.loglike(thisParameter.values)

    worse_col = []
    bigger_pvalue = 0
    LLK0_saved = 0
    DF0_saved = 0
    Dev_saved = 0
    DF_saved = 0
    AIC_saved = 0
    BIC_saved = 0
    
    
    for col in columns:

        X2 = X.drop(col, axis = 1)
        X2 = stats.add_constant(X2, prepend=True)

        DF0 = np.linalg.matrix_rank(X2) * (len(y.value_counts()) - 1)

        logit = stats.MNLogit(y, X2)
        thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
        thisParameter = thisFit.params
        LLK0 = logit.loglike(thisParameter.values)

        Deviance = 2 * (LLK1 - LLK0)
        DF = DF1 - DF0
        pValue = scipy.stats.chi2.sf(Deviance, DF)
        
        MDF = (thisFit.J - 1) * thisFit.K
        LLK = thisFit.llf

        NSample = len(y)
        AIC = 2.0 * MDF - 2.0 * LLK
        BIC = MDF * np.log(NSample) - 2.0 * LLK

        if pValue > bigger_pvalue:
            worse_col = col
            bigger_pvalue = pValue
            LLK0_saved = LLK0
            DF0_saved = DF0
            Dev_saved = Deviance
            DF_saved = DF
            AIC_saved = AIC
            BIC_saved = BIC

    
    if bigger_pvalue < 0.05:
        best_model = X2.columns
        print('\n Best model --> Removed feature: ', removed_columns)
        print("Model Log-Likelihood Value =", LLK0_saved)
        print("Number of Free Parameters =", DF0_saved)
        print("Deviance (Statistic, DF, Significance)", Dev_saved, DF_saved, bigger_pvalue)
        print("Akaike Information Criterion =", AIC_saved)
        print("Bayesian Information Criterion =", BIC_saved)
        print('\n')
        break
    
    removed_columns.append(worse_col)
    print('\nModel', i, ' --> Removed feature: ', removed_columns)
    print("Model Log-Likelihood Value =", LLK0_saved)
    print("Number of Free Parameters =", DF0_saved)
    print("Deviance (Statistic, DF, Significance)", Dev_saved, DF_saved, bigger_pvalue)
    print("Akaike Information Criterion =", AIC_saved)
    print("Bayesian Information Criterion =", BIC_saved)    
    print('\n')
    i += 1

Optimization terminated successfully.
         Current function value: 0.195606
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.222527
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.195633
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.195628
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.578049
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.195647
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.195808
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.195607
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.195811
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.1

Optimization terminated successfully.
         Current function value: 0.811988
         Iterations 7

 Best model --> Removed feature:  ['x7', 'x3', 'x2', 'x5', 'x9', 'x6', 'x8']
Model Log-Likelihood Value = -2230.4083138546557
Number of Free Parameters = 6
Deviance (Statistic, DF, Significance) 537.0114060078226 2 2.4516294286009425e-117
Akaike Information Criterion = 4472.816627709311
Bayesian Information Criterion = 4516.078669941168




In [93]:
y = df['y']
X2 = df.drop(['y'], axis=1)
X2 = X.drop(removed_columns, axis=1)
X2 = stats.add_constant(X2, prepend=True)

DF1 = np.linalg.matrix_rank(X2) * (len(y.value_counts()) - 1)

logit = stats.MNLogit(y, X2)
thisFit = logit.fit(method='newton', full_output = True, maxiter = 100, tol = 1e-8)
thisParameter = thisFit.params
LLK1 = logit.loglike(thisParameter.values)

print(thisFit.summary())
print("Model Log-Likelihood Value =", LLK1)
print("Number of Free Parameters =", DF1)

Optimization terminated successfully.
         Current function value: 0.196190
         Iterations 10
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:                10000
Model:                        MNLogit   Df Residuals:                     9992
Method:                           MLE   Df Model:                            6
Date:                Tue, 02 Mar 2021   Pseudo R-squ.:                  0.8164
Time:                        23:50:35   Log-Likelihood:                -1961.9
converged:                       True   LL-Null:                       -10688.
Covariance Type:            nonrobust   LLR p-value:                     0.000
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0168      0.087     11.675      0.000       0.846       1.188
x1            -1.1145      0

In [94]:
for col in columns:

    X2 = X.drop(col, axis = 1)
    X2 = stats.add_constant(X2, prepend=True)
    
    objLogit = stats.MNLogit(y, X2)
    thisFit = objLogit.fit(method = 'newton', maxiter = 100, tol = 1e-8)

    MDF = (thisFit.J - 1) * thisFit.K
    LLK = thisFit.llf
    
    NSample = len(y)
    AIC = 2.0 * MDF - 2.0 * LLK
    BIC = MDF * np.log(NSample) - 2.0 * LLK
    
    print('\nModel 1 --> Removed feature: ', col)
    #print(thisFit.summary())
    print("Akaike Information Criterion =", AIC)
    print("Bayesian Information Criterion =", BIC)
    print('\n')

Optimization terminated successfully.
         Current function value: 0.222527
         Iterations 10

Model 1 --> Removed feature:  x1
Akaike Information Criterion = 4490.542699405585
Bayesian Information Criterion = 4634.749506845109


Optimization terminated successfully.
         Current function value: 0.195633
         Iterations 10

Model 1 --> Removed feature:  x2
Akaike Information Criterion = 3952.662121276995
Bayesian Information Criterion = 4096.868928716519


Optimization terminated successfully.
         Current function value: 0.195628
         Iterations 10

Model 1 --> Removed feature:  x3
Akaike Information Criterion = 3952.560725487029
Bayesian Information Criterion = 4096.767532926552


Optimization terminated successfully.
         Current function value: 0.578049
         Iterations 9

Model 1 --> Removed feature:  x4
Akaike Information Criterion = 11600.988729969893
Bayesian Information Criterion = 11745.195537409416


Optimization terminated successfully.
     

In [97]:
X2 = X.drop(removed_columns, axis = 1)
X2 = stats.add_constant(X2, prepend=True)

objLogit = stats.MNLogit(y, X2)
thisFit = objLogit.fit(method = 'newton', maxiter = 100, tol = 1e-8)

MDF = (thisFit.J - 1) * thisFit.K
LLK = thisFit.llf

NSample = len(y)
AIC = 2.0 * MDF - 2.0 * LLK
BIC = MDF * np.log(NSample) - 2.0 * LLK

print('\nModel 1 --> Removed feature: ', removed_columns)
#print(thisFit.summary())
print("Akaike Information Criterion =", AIC)
print("Bayesian Information Criterion =", BIC)
print('\n')

Optimization terminated successfully.
         Current function value: 0.196190
         Iterations 10

Model 1 --> Removed feature:  ['x2', 'x3', 'x5', 'x6', 'x7', 'x8', 'x9']
Akaike Information Criterion = 3939.805221701489
Bayesian Information Criterion = 3997.4879446772984


